# Week 5 ARIMA vs Prophet
## Time Series Analysis
### Earnest Salgado
### ID 12285784

In [1]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from sktime.forecasting.all import ForecastingHorizon
from sktime.forecasting.all import ExponentialSmoothing
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

/Users/earnestsalgado/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/earnestsalgado/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/earnestsalgado/opt/anaconda3/lib/python3.9/site-packages/sktime/utils/validation/_dependencies.py:64: UserWarning: No module named 'prophet'. 'prophet' is a soft dependency and not included in the sktime installation. Please run: `pip install prophet` to install the prophet package. To install all soft dependencies, run: `pip install sktime[

## 1. 10 pts - Exploratory Data Analysis

### 1.A 5 pts Merge the data sets together, should have 735 rows of data

In [66]:
df_co2 = pd.read_csv('hw5_data_co2.csv', parse_dates = ['ds'])
df_temp = pd.read_csv('hw5_data_temp.csv', parse_dates = ['ds'])

In [17]:
print(df_co2.isnull().sum())
print(df_temp.isnull().sum())

print(df_co2.shape)
print(df_temp.shape)

ds     0
co2    0
dtype: int64
ds      0
temp    0
dtype: int64
(735, 2)
(1674, 2)


In [36]:
df_temp

,ds,temp
0,1880-01-01,56.40
1,1880-02-01,56.82
2,1880-03-01,56.74
3,1880-04-01,56.55
4,1880-05-01,56.85
...,...,...
1669,2019-02-01,58.37
1670,2019-03-01,58.59
1671,2019-04-01,58.50
1672,2019-05-01,58.50


In [37]:
df_co2

,ds,co2
0,1958-03-15,315.700
1,1958-04-15,317.450
2,1958-05-15,317.510
3,1958-06-15,316.685
4,1958-07-15,315.860
...,...,...
730,2019-01-15,410.920
731,2019-02-15,411.660
732,2019-03-15,412.000
733,2019-04-15,413.510


In [18]:
df_co2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      735 non-null    datetime64[ns]
 1   co2     735 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 11.6 KB


In [42]:
df_co2['ds'].dt.month

0      3
1      4
2      5
3      6
4      7
      ..
730    1
731    2
732    3
733    4
734    5
Name: ds, Length: 735, dtype: int64

In [67]:
df = pd.merge(df_co2, df_temp, 
              left_on = [df_co2['ds'].dt.year, df_co2['ds'].dt.month],
              right_on = [df_temp['ds'].dt.year, df_temp['ds'].dt.month],
              how = 'right', suffixes = ("_co2", "_temp"),
             ).drop(["key_0", "key_1"], axis=1)

In [69]:
df

,ds_co2,co2,ds_temp,temp
0,NaT,NaN,1880-01-01,56.40
1,NaT,NaN,1880-02-01,56.82
2,NaT,NaN,1880-03-01,56.74
3,NaT,NaN,1880-04-01,56.55
4,NaT,NaN,1880-05-01,56.85
...,...,...,...,...
1669,2019-02-15,411.66,2019-02-01,58.37
1670,2019-03-15,412.00,2019-03-01,58.59
1671,2019-04-15,413.51,2019-04-01,58.50
1672,2019-05-15,414.83,2019-05-01,58.50


In [70]:
df.set_index('ds_temp', inplace = True)

In [71]:
df

,ds_co2,co2,temp
ds_temp,,,
1880-01-01,NaT,NaN,56.40
1880-02-01,NaT,NaN,56.82
1880-03-01,NaT,NaN,56.74
1880-04-01,NaT,NaN,56.55
1880-05-01,NaT,NaN,56.85
...,...,...,...
2019-02-01,2019-02-15,411.66,58.37
2019-03-01,2019-03-15,412.00,58.59
2019-04-01,2019-04-15,413.51,58.50


In [72]:
df = df.drop(["ds_co2"], axis=1)

In [73]:
df

,co2,temp
ds_temp,,
1880-01-01,NaN,56.40
1880-02-01,NaN,56.82
1880-03-01,NaN,56.74
1880-04-01,NaN,56.55
1880-05-01,NaN,56.85
...,...,...
2019-02-01,411.66,58.37
2019-03-01,412.00,58.59
2019-04-01,413.51,58.50
